In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import hypothesis as h
from plotting_tools import make_dirs_safe

import time
plt.rcParams['figure.figsize'] = 10, 5

# Analysis of the rank of the constrain matrix

In this notebook we use a little rearanged definitions of the left and right parts of matrix:

Let us write $\pmb{f_n}^\top$ as $[1\ \pmb{g_n}^\top]^\top$, and similarly
$\tilde{\pmb{a}}_m^\top = [\pmb{a_m}^\top 1]^\top$. We can then 
write the whole constraint vector as:
$$\begin{bmatrix}
\text{vect}(\tilde{\pmb{a}}_m\pmb{f}_n^\top)^\top & \text{vect}( 
\pmb{g_n} \pmb{f_n}^\top)^\top
\end{bmatrix}$$
Then we have the needed constrains on the left, and constrains added due to the 
regularisation on the right.
From the Lemmas from the paper we know that we can write:
$$\text{vect}(\pmb{f_n f_n^\top}) = \pmb{R} \pmb{f_n^e},$$
where $\pmb{R}$ is some (sparse) matrix, and $\pmb{f_n^e}$ is a vector 
similar in structure to $\pmb{f_n}$, and can be written as a concatenation of  $\pmb{f_n}$ and $\pmb{f}_n^{r}$.
This means we can write our full system of equations as:
\begin{align}
b &=
\begin{bmatrix}
\text{vect}(\pmb{a_m f_n}^\top)^\top & (\pmb{f_n^e})^\top
\end{bmatrix}
\begin{bmatrix}
\pmb{C} \\
\pmb{R^\top L} \\\end{bmatrix}\\
\end{align}
Or using $\tilde{\pmb{a}}_m$:
\begin{align}
b &=
\begin{bmatrix}
\text{vect}(\tilde{\pmb{a}}_m \pmb{f}_n^\top)^\top & \pmb{f}_n^{r\top}
\end{bmatrix}
\begin{bmatrix}
\pmb{C} \\
\pmb{R}^\top \pmb{L} \\\end{bmatrix}
\end{align}

And so for us left hand side refers to $\text{vect}(\tilde{\pmb{a}}_m \pmb{f}_n^\top)^\top$ and right hand side refers to $\pmb{f}_n^{r\top}$.


## Rank vs number of measurements
In those experiments, the number of dimentions $D$, number of constrains $K$ and number of positions $N$ is fixed,
and for several different ranks the number of measurements is increased.

In [ ]:
experiment_params={
    "n_dimensions": 2,
    "n_constraints": 5,
    "n_positions": 20,
    "min_positions": 10,
    "n_repetitions": 100,
    "full_matrix": True,
    "n_anchors_list": [1, 2, 4, 8, 100],
}

ranks, params = h.matrix_rank_experiment(experiment_params)

plt.rcParams['figure.figsize'] = 10, 5
h.plot_results(ranks, params)
plt.show()

### Rank vs number of positions
Here the number of dimentions $D$, the number of constrains $K$ and the total number of measrurements are fixed.
In particular, the number of measurements is $(D+1)K$, and we increase total number of sampling positions along the trajectory. We can see drastically different behaviour for exactly $D+1$ anchors than for more than $D+1$ anchors.

In [ ]:
experiment_params={
    "n_dimensions": 2,
    "n_constraints": 5,
    "fixed_n_measurements": 0,
    "min_positions": 7,
    "max_positions": 100,
    "n_repetitions": 1000,
    "full_matrix": True,   
    "n_anchors_list": [1, 2, 4, 8, 100],
}

start = time.time()
ranks, params = h.matrix_rank_experiment(experiment_params)

plt.rcParams['figure.figsize'] = 10, 5
h.plot_results(ranks, params,  save=True)
plt.show()
end = time.time()
print("elapsed time: {:.2f}s".format(end - start))

In [ ]:
params["fixed_n_measurements"]

In [ ]:
experiment_params={
    "n_dimensions": 2,
    "n_constraints": 5,
#     "fixed_n_measurements": 2*(2+1)*5,
    "fixed_n_measurements": 20,
    "min_positions": 7,
    "max_positions": 100,
    "n_repetitions": 1000,
    "full_matrix": True,   
    "n_anchors_list": [1, 2, 4, 8, 100],
}

start = time.time()
ranks, params = h.matrix_rank_experiment(experiment_params)
end = time.time()
print("elapsed time: {:.2f}s".format(end - start))

In [ ]:
plt.rcParams['figure.figsize'] = 10, 5
h.plot_results(ranks, params, save=True)
plt.show()
params["fixed_n_measurements"]

In [ ]:
experiment_params={
    "n_dimensions": 2,
    "n_constraints": 5,
    "fixed_n_measurements": 2*(2+1)*5,
    "min_positions": 7,
    "max_positions": 100,
    "n_repetitions": 1000,
    "full_matrix": True,   
    "n_anchors_list": [1, 2, 4, 8, 100],
}

start = time.time()
ranks, params = h.matrix_rank_experiment(experiment_params)
end = time.time()
print("elapsed time: {:.2f}s".format(end - start))

In [ ]:
plt.rcParams['figure.figsize'] = 10, 5
h.plot_results(ranks, params, save=True)
plt.show()
params["fixed_n_measurements"]

## Theory vs experiments

### $D+1$ anchros 

For exactly $D+1$ anchors, each anchor is 
"responsible" for a direction. Indeed, if we change the frame to the basis 
defined by our anchors, it will change the coefficients $\pmb{C}$ we want to 
recover, but won't change $\pmb{f}_n$s. But then the constrain matrix will 
have a form:
$$\begin{bmatrix}
\pmb{f_{n_1}} & \pmb{0} & \pmb{0} \\
\pmb{0} & \pmb{f_{n_2}} & \pmb{0} \\
\pmb{0} & \pmb{0} & \pmb{f_{n_3}}\\
\pmb{f_{n_4}} & \pmb{0} & \pmb{0} \\
\pmb{0} & \pmb{f_{n_5}} & \pmb{0} \\
\vdots & \vdots & \vdots \\ 
\pmb{0} & \pmb{0} & \pmb{f_{n_{(D+1)K}}}\\
\end{bmatrix},
$$
where $\pmb{0}$ is a vector of zeros of length $K$. Then we can see that 
in order to diagonalise the matrix, we need at least $K$ different 
$\pmb{f}_n$s for each $K$. Since $\pmb{f_n}$s do not repeat for the 
same anchor, this is sufficient and necessary condition. 

We can then calculate what is the probability of such situation. Assume that  
we have $R = (D+1)K$ measurements, $M = D+1$ anchors and $N$ possible 
$f_n$s. We can pick $(D+1)K$ measurements out of $(D+1)N$ in:
$${(D+1)N}\choose{(D+1)K}$$
ways. On the other hand, there is:
$${{N}\choose{K}}^{D+1}$$
ways choose the $K$ measurements corresponding to each anchor.
So the probability of having the right number of measurements for each anchor 
is:
$$
P_{N} = {{{N}\choose{K}}^{D+1}}\bigg/ {{{(D+1)N}\choose{(D+1)K}}}
$$
Note that for $N=K$ we just get $P_{K}=1$.

Using approximation formulas from Wikipedia, get a limit for large $N$:
\begin{align*}
P_{N\rightarrow \infty} &\approx \frac{\sqrt{D+1}}{\sqrt{2\pi 
K}^{D}}
\end{align*}

### More thatn $D+1$ anchors

In [ ]:
experiment_params={
    "n_dimensions": 2,
    "n_constraints": 5,
    "fixed_n_measurements": 0,
    "min_positions": 5,
    "max_positions": 50,
    "n_repetitions": 1000,
    "full_matrix": True,
#     "n_anchors_list": [3]
    "n_anchors_list": [1, 2, 3, 4],
}

print("max rank:", 8)
start = time.time()
ranks, params = h.matrix_rank_experiment(experiment_params)
print(params["n_anchors_list"])

plt.rcParams['figure.figsize'] = 10, 5
h.plot_results(ranks, params)
plt.show()
end = time.time()
print("elapsed time: {:.2f}s".format(end - start))

In [ ]:
n_dimensions = 2
n_constrains = 5
n_positions_list = params["second_list"]
print(params["fixed_n_measurements"])

idx = 1
n_anchors = params["n_anchors_list"][idx]
probabilities1 = [h.probability_upper_bound_any_measurements(
    n_dimensions,
    n_constrains,
    n,
    position_wise=False,
    n_anchors=n_anchors,
    n_measurements=params["fixed_n_measurements"]
) for n in n_positions_list]
probabilities2 = [h.probability_upper_bound_any_measurements(
    n_dimensions,
    n_constrains,
    n,
    position_wise=True,
    n_anchors=n_anchors,
    n_measurements=params["fixed_n_measurements"]
) for n in n_positions_list]
# probabilities3 = [h.probability_lower_bound(n_dimensions, n_constrains, n, n_anchors=n_dimensions + 1 + idx) for n in n_positions_list]
probabilities1 = np.array(probabilities1)
probabilities2 = np.array(probabilities2)

In [ ]:
max_rank = params["max_rank"]
mean =  np.mean(ranks[:, idx, :] >= max_rank, axis=-1)
std = np.std(ranks[:, idx, :] >= max_rank, axis=-1)

f, ax = plt.subplots()
beg = 0

plt.plot(
    n_positions_list[beg:],
    mean[beg:],
    label="estimated probability")
plt.plot(
    n_positions_list[beg:],
    probabilities1[beg:],
    label="first upper bound")
plt.plot(
    n_positions_list[beg:],
    probabilities2[beg:],
    label="second upper bound")
# plt.plot(
#     n_positions_list[beg:crop],
#     np.multiply(probabilities2[beg:], probabilities[beg:crop]),
#     label="both")

plt.xlabel("number of positions")
plt.ylim(0)
plt.grid()
plt.legend()
plt.title("Upper bounds for D + {} anchors".format(n_anchors-params["n_dimensions"]))
plt.savefig(params["directory"] + str(n_anchors) + "full_matrix.pdf")
plt.show()
